In [1]:
import os
import glob
from src.thumbnails_extractor import extract_frames_every_n_seconds_dir
from pathlib import Path

In [ ]:
#####################################
# Download the dataset

# The main page link is here: https://serre-lab.clps.brown.edu/resource/breakfast-actions-dataset/
# The corresponding download link is here: https://drive.google.com/open?id=1jgSoof1AatiDRpGY091qd4TEKF-BUt6I
# To download from the command line
# ```
# pip install gdown
# gdown https://drive.google.com/open?id=1jgSoof1AatiDRpGY091qd4TEKF-BUt6I --fuzzy 
# ```
# If it is too suspicious to download from the random link, navigate to the main page link instead.

In [ ]:
#####################################
# Decompress the dataset

!tar -xzvf data/BreakfastII_15fps_qvga_sync.tar.gz -C data

In [ ]:
# Parameters
decompressed_dir = "data/BreakfastII_15fps_qvga_sync"
thumbnails_base_dir = "data/Breakfast/thumbnails"
groundtruth_dir = "data/Breakfast/groundtruth"
sampling_fps = 1
resolution=(320,240)

In [ ]:
#####################################
# Flatten the folder structure


# Find all files recursively
for filepath in glob.glob(os.path.join(decompressed_dir, "**", "*"), recursive=True):
	if os.path.isfile(filepath):  # Ensure it's a file
		
		# Change relative path
		relative_path = os.path.relpath(filepath, decompressed_dir)
		new_relative_path = relative_path.replace("/", "_")
		new_path = os.path.join(decompressed_dir, new_relative_path)
		
		os.rename(filepath, new_path)


In [ ]:
#####################################
# Extract the thumbnails

# Make new folder to store 
os.makedirs(thumbnails_base_dir,exist_ok=True)

# Extract thumbnails
extract_frames_every_n_seconds_dir(videos_parent_dir = decompressed_dir,
									thumbnails_output_parent_dir = thumbnails_base_dir,
									sampling_fps=sampling_fps, 
									thumbnails_resolution=resolution)


In [ ]:
#####################################
# Generate the groundtruth files

def frame_nb_in_labels(frame_nb, labels_dict):
	for label, (lower_bound, upper_bound) in labels_dict.items():
		if lower_bound <= frame_nb and frame_nb <= upper_bound:
			return label 
	return "NIL"

# Make the folder
os.makedirs(groundtruth_dir,exist_ok=True)

# Read the thumbnails folder first
thumbnails_dir = f"{thumbnails_base_dir}/thumbnails_{sampling_fps}secsPerFrame_{resolution[0]}px{resolution[1]}px"
video_ids = os.listdir(thumbnails_dir)
video_ids.sort()

for video_id in video_ids: 

	# For each video_id, find the corresponding labels file
	groundtruth_path = f"{decompressed_dir}/{video_id}.avi.labels"

	try:
		# Get labels dictionary
		labels_dict = dict()
		with open(groundtruth_path,"r") as file:
			for line in file:
				frame_intervals, label = line.strip().split(" ")
				frame_lower_bound, frame_upper_bound = frame_intervals.split("-")
				labels_dict[label] = (int(frame_lower_bound),int(frame_upper_bound))

		# Open thumbnails and loop through
		frame_files = os.listdir(f"{thumbnails_dir}/{video_id}")
		frame_nbs = [int(Path(file).stem) for file in frame_files]
		frame_nbs.sort()

		# Generate groundtruth labels
		groundtruth_file = []
		for frame_nb in frame_nbs:
			groundtruth_file.append(frame_nb_in_labels(frame_nb, labels_dict))

		# Write groundtruth file
		with open(f"{groundtruth_dir}/{video_id}.txt","w") as file:
			file.writelines(f"{item}\n" for item in groundtruth_file)

	except:
		print(f"Label for {video_id} not avail")



In [ ]:
######################
# Initiate the training

from src.train_and_infer import initialise_training

initialise_training(cfg_path="src/configs/default.yaml",
                    thumbnails_dir=thumbnails_dir,
                    groundtruth_dir=groundtruth_dir)



In [ ]:
######################
# Infer using a new directory
#
# The performance of the model is saved in the log directory
# log/[initiated_training_timestamp]/train_metrics.csv : contains the training metrics
# log/[initiated_training_timestamp]/test_metrics.csv : contains the evaluation metrics
# log/[initiated_training_timestamp]/model_weights : contains the model weights 

from src.train_and_infer import infer_with_new_dir

infer_with_new_dir(cfg_path="log/2025Mar03_1414h/log_config.yaml",
                   model_weights_path="log/2025Mar03_1414h/model_weights/network_iter_29.net",
                   eval_output_dir="log/2025Mar03_1414h",
                   dataset_name="new_test",
                   thumbnails_dir=thumbnails_dir,
                   groundtruth_dir=None,
                   video_list=["P41_stereo_P41_friedegg_ch0"])